```
Name: Andrea Fox
Date: May 8, 2021
Course: DSC650 - T301 - Big Data
Assignment: Assignment 08 - Producer
```

In [55]:
#load libraries
import json
import uuid

import os
import pandas as pd
import datetime
import threading
from pathlib import Path
import time
import s3fs
import pyarrow.parquet as pq
from collections import namedtuple

from kafka import KafkaProducer, KafkaAdminClient
from kafka.admin.new_topic import NewTopic
from kafka.errors import TopicAlreadyExistsError

In [56]:
#load data
endpoint_url='https://storage.budsc.midwest-datascience.com'
s3 = s3fs.S3FileSystem(
    anon=True,
    client_kwargs={
        'endpoint_url': endpoint_url
    }
)

In [57]:
#configuration parameters copied from example
config = dict(
    bootstrap_servers=['kafka.kafka.svc.cluster.local:9092'],
    first_name='Andrea',
    last_name='Fox'
)

config['client_id'] = '{}{}'.format(
    config['last_name'], 
    config['first_name']
)
config['topic_prefix'] = '{}{}'.format(
    config['last_name'], 
    config['first_name']
)

config

{'bootstrap_servers': ['kafka.kafka.svc.cluster.local:9092'],
 'first_name': 'Andrea',
 'last_name': 'Fox',
 'client_id': 'FoxAndrea',
 'topic_prefix': 'FoxAndrea'}

In [58]:
#create Kafka topic function
def create_kafka_topic(topic_name, config=config, num_partitions=1, replication_factor=1):
    bootstrap_servers = config['bootstrap_servers']
    client_id = config['client_id']
    topic_prefix = config['topic_prefix']
    name = '{}-{}'.format(topic_prefix, topic_name)
    
    admin_client = KafkaAdminClient(
        bootstrap_servers=bootstrap_servers, 
        client_id=client_id
    )
    
    topic = NewTopic(
        name=name,
        num_partitions=num_partitions,
        replication_factor=replication_factor
    )

    topic_list = [topic]
    try:
        admin_client.create_topics(new_topics=topic_list)
        print('Created topic "{}"'.format(name))
    except TopicAlreadyExistsError as e:
        print('Topic "{}" already exists'.format(name))
    
create_kafka_topic('locations')
create_kafka_topic('accelerations')

Topic "FoxAndrea-locations" already exists
Topic "FoxAndrea-accelerations" already exists


In [59]:
#creates KafkaProducer object
producer = KafkaProducer(
  bootstrap_servers=config['bootstrap_servers'],
  value_serializer=lambda x: json.dumps(x).encode('utf-8')
)

In [60]:
#send data function
def on_send_success(record_metadata):
    print('Message sent:\n    Topic: "{}"\n    Partition: {}\n    Offset: {}'.format(
        record_metadata.topic,
        record_metadata.partition,
        record_metadata.offset
    ))
    
def on_send_error(excp):
    print('I am an errback', exc_info=excp)
    # handle exception

def send_data(topic, data, config=config, producer=producer, msg_key=None):
    topic_prefix = config['topic_prefix']
    topic_name = '{}-{}'.format(topic_prefix, topic)
    
    if msg_key is not None:
        key = msg_key
    else:
        key = uuid.uuid4().hex
    
    producer.send(
        topic_name, 
        value=data,
        key=key.encode('utf-8')
    ).add_callback(on_send_success).add_errback(on_send_error)

In [61]:
example_data = dict(
    key1='value1',
    key2='value2'
)

send_data('locations', example_data)

In [62]:
#load accelerations
acceleration_columns = [
    'offset',
    'id',
    'ride_id',
    'uuid',
    'x',
    'y',
    'z',
     't'
]
Acceleration = namedtuple('Acceleration', acceleration_columns)

df_acc = pq.ParquetDataset('s3://data/processed/bdd/accelerations',filesystem=s3).read_pandas().to_pandas()
df_acc = df_acc[acceleration_columns].sort_values(by=['offset'])

Message sent:
    Topic: "FoxAndrea-locations"
    Partition: 0
    Offset: 486


In [63]:
#load locations
location_columns = [
    'offset',
    'id',
    'ride_id',
    'uuid',
    'course',
    'latitude',
    'longitude',
    'geohash',
    'speed',
    'accuracy',
     't'
]
Location = namedtuple('Location', location_columns)

df_loc = pq.ParquetDataset('s3://data/processed/bdd/locations',filesystem=s3).read_pandas().to_pandas()
df_loc = df_loc[location_columns].sort_values(by=['offset'])

In [64]:
#creating 3 separate lists for both accelerations and locations
start_time = time.time()

offset = []
topics = []
datas = []

for i in range(df_acc.shape[0]):
    data_acc = df_acc.loc[i].to_dict()
    offset.append(data_acc['offset'])
    datas.append(data_acc)
    topics.append('accelerations')
    
for i in range(df_loc.shape[0]):
    data_loc = df_loc.loc[i].to_dict()
    offset.append(data_loc['offset'])
    datas.append(data_loc)
    topics.append('locations')

In [65]:
#combining the lists into a dataframe and sorting by offset
df = pd.DataFrame({'offset': offset, 'topics': topics, 'datas': datas})
df = df.sort_values(by = ['offset'])
df.head()

offset         topics                                              datas
0  0.822061  accelerations  {'offset': 0.8220608865228429, 'id': '58682c5d...
1  0.842061  accelerations  {'offset': 0.8420608865228429, 'id': '58682c5d...
2  0.862061  accelerations  {'offset': 0.862060886522843, 'id': '58682c5d4...
3  0.882061  accelerations  {'offset': 0.882060886522843, 'id': '58682c5d4...
4  0.902061  accelerations  {'offset': 0.9020608865228429, 'id': '58682c5d...

In [66]:
for i in range(df.shape[0]):
    offset = df['offset'][i]
    while (time.time() - start_time) < offset:
        pass
    send_data(df['topics'][i], df['datas'][i])

Message sent:
    Topic: "FoxAndrea-accelerations"
    Partition: 0
    Offset: 23512
Message sent:
    Topic: "FoxAndrea-accelerations"
    Partition: 0
    Offset: 23513
Message sent:
    Topic: "FoxAndrea-accelerations"
    Partition: 0
    Offset: 23514
Message sent:
    Topic: "FoxAndrea-accelerations"
    Partition: 0
    Offset: 23515
Message sent:
    Topic: "FoxAndrea-accelerations"
    Partition: 0
    Offset: 23516
Message sent:
    Topic: "FoxAndrea-accelerations"
    Partition: 0
    Offset: 23517
Message sent:
    Topic: "FoxAndrea-accelerations"
    Partition: 0
    Offset: 23518
Message sent:
    Topic: "FoxAndrea-accelerations"
    Partition: 0
    Offset: 23519
Message sent:
    Topic: "FoxAndrea-accelerations"
    Partition: 0
    Offset: 23520
Message sent:
    Topic: "FoxAndrea-accelerations"
    Partition: 0
    Offset: 23521
Message sent:
    Topic: "FoxAndrea-accelerations"
    Partition: 0
    Offset: 23522
Message sent:
    Topic: "FoxAndrea-accelerations"
   